<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../../README.md)

7.1 : Find Influencers on Twitter
================================

### Overview
Find 'influencers' on Twitter graph

### Depends On
None

### Run time
20 mins

### Lab Setup

This lab uses the GraphFrames spark package.  It is currently NOT part of the default spark framework.

We use this for two reasons:

1. GraphFrames likely will be the basis of future graph processing in spark.

2. RDD-based graphx has no python API, whereas GraphFrames does have a python API. As this jupyter notebook is python that makes it required.

So, to run this we have to start pyspark like this:

```bash
PYSPARK_PYTHON=python3 PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/spark/bin/pyspark --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11
```

Replace the version on graphframes with your latest.

With that said, let's start:

## Step 1: Build the following twitter graph
Here some real world data:
<img src="../assets/images/7.1a.png" style="border: 5px solid grey; max-width:100%;"/>

All of the following steps are performed by entering the commands in the Spark Scala shell

#### Import the necessary libraries

In [ ]:
from graphframes import *

#### Construct the array of vertices

 Data structure: twitter handle, number of followers, gender of the tweeter

In [ ]:
vertexArray = sqlContext.createDataFrame([
        (1, "@markkerzner", 309, "M"),  # (Name, # followers, gender)
        (2, "@mjbrender", 3101, "M"),
        (3, "@dridisahar1", 27, "F"),
        (4, "@dez_blanchfield ", 38600, "M"),
        (5, "@ch_doig ", 519, "F"),
        (6, "@Sunitha_Packt ", 332, "F"),
        (7, "@WibiData ", 2477, "N")  # company, so gender neutral
], ["id", "Name", "followers", "gender"]) 

vertexArray.show()

####  Construct the array of edges

On this step, these are all my followers, so they connect to me

In [ ]:
edgeArray = sqlContext.createDataFrame([
        (1, 2, 7), # src,  dest,   # retweets
        (1, 3, 2),
        (1, 4, 4),
        (1, 5, 3),
        (1, 6, 1),
        (1, 7, 2)
], ["src", "dst", "retweets"])
edgeArray.show()

We are using data from a real Twitter account, if you want, you can use yours

<img src="../assets/images/7.1-twitter.png" style="border: 5px solid grey; max-width:100%;"/>

#### Prepare the data for Spark processing. What do ".parallelize" methods below accomplish?

#### Construct the graph from the vertices and edges

In [ ]:
graph = GraphFrame(vertexArray, edgeArray)

## Step 3 : Analyzing Graph
#### Print graph

In [ ]:
# Vertices
graph.vertices.show()


# Edges
graph.edges.show()

#graph.vertices.foreach { case (id, (name, nFollow, gender)) =>
#    println(s"Tweeter $name has $nFollow followers and is $gender") }

# triplets
#    graph.triplets.foreach(println)

#### Filter out male followers

In [ ]:
graph.vertices.filter("gender != 'M'").show()

#### Find my significant followers

In [ ]:
graph.vertices.filter("followers > 1000").show() 

#### Find those followers who do enough re-tweeets for me

In [ ]:
graph.edges.filter("retweets > 5").show()

graph.edges.filter("retweets > 5").count()

#### Count my male and female followers

In [ ]:
num_male = graph.vertices.filter("gender == 'M'").count()
num_female = graph.vertices.filter("gender == 'F'").count()
print ('Males %d, Females %d' % (num_male, num_female))